In [ ]:
#==================================================================
#Program: AnnotationTransfer
#Version: 1.0
#Author(s): David Helminiak
#Date Created: 20 June 2025
#Date Last Modified: 20 August 2025
#Description: Form corrected/uncorrected WSI/patch sets as would be produced by DDSM-GUI v2.1.3+ using annotated stitched/corrected WSI for reference
#Note: Must manually install imageJ/Fiji (JRE rec.) and the BaSiC plugin (not available directly through Maven repository)
#      Must have MATLAB Python API setup
#      Will almost certainly have OOM somewhere after the halfway mark, with only 64GB RAM available on system

#Malignant: Patch contains ANY locations designated as malignant 
#Edge:      Patch contains ANY locations not in the foreground
#Boundary:  Patch contains both malignant and benign locations

#Grid overlay color guide:
#0 Lime    - Benign
#2 Cyan    - Benign, Edge
#4 Red     - Malignant
#5 Orange  - Malignant, Boundary
#6 Magenta - Malignant, Edge
#7 Yellow  - Malignant Boundary, Edge

#Shouldn't see benign and boundary cases (blue, white); if there is malignant tissue, then that is the principal label
#1 Blue    - Benign, Boundary
#3 White   - Benign, Edge, Boundary

#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#Specify input directory references
#cWSI/aWSI should contain one .jpg per sample (e.g. 72_1.jpg)
#uBlocks should contain one folder of block images per sample (e.g. 72_1/72_1_L01_R01_C01.tif, 72_1/72_1_L01_R01_C02.tif, etc.)
dir_in_cWSI = './INPUTS/WSI_CORRECTED/'
dir_in_aWSI = './INPUTS/WSI_ANNOTATIONS/'
dir_in_uBlocks = './INPUTS/BLOCKS_UNCORRECTED/'

#Specify samples that are wholly benign or malignant

#Dataset_1
#samplesMixed = [3_1, 6_1, 10_3, 12_1, 13_1, 14_2, 15_4, 21_1, 32_2, 38_1, 42_3, 43_1, 48_3, 49_1, 52_2, 53_2, 55_2, 59_2, 66_2]
#samplesBenign = ['2_1', '7_2', '8_1', '11_3', '16_3', '19_2', '20_3', '22_3', '23_3', '24_2', '25_3', '26_3', '27_1', 
#                 '28_2', '29_2', '30_2', '44_1', '47_2', '57_2', '58_2', '67_1', '68_1', '69_1', '70_1'
#                ]
#samplesMalignant = ['4_4', '5_3', '9_3', '17_5', '31_1', '33_3', '34_1', '35_4', '36_2', '37_1', '40_2', '45_1', '46_2', 
#                    '50_1', '51_2', '54_2', '56_2', '60_1', '61_1', '62_1', '63_3', '64_1', '65_1'
#                   ]
#failedBlockFilenames = [? - Not recorded]
#discardBlockFilenames = [None]

#Dataset_2
#samplesMixed = [? - Not recorded]
#samplesBenign = ['72_1', '74_1', '80_1', '83_1', '96_1', '98_1', '100_1', '118_1', '119_1', '120_1', '122_2', '123_1',
#                '124_1', '126_2', '127_1', '128_2', '129_1', '130_1', '131_1', '134_2', '137_2', '138_1', '140_1', '141_1',
#                '143_1', '144_1', '145_1', '146_1', '149_1', '154_1', '165_1', '169_1', '172_1', '173_1'
#                ]
#samplesMalignant = ['75_1', '84_1', '88_1', '91_1', '99_1', '101_1', '105_1']
#failedBlockFilenames = ['11_3_L01_R03_C08.tif', '11_3_L01_R06_C04.tif']
#discardBlockFilenames = ['16_3_L01_R09_C09.tif', '22_3_L01_R03_C12.tif', '28_2_L01_R13_C09.tif', '28_2_L01_R14_C01.tif', '28_2_L01_R14_C08.tif', '28_2_L01_R14_C09.tif']

#If only some of the available samples should be considered (second pass for samples with blocks that failed to match correctly), add them here
#Make sure to move any correctly completed samples out of the output directory, as they will otherwise get erased during the initial setup
manualSamples = []

#If any potentially available samples should be skipped
sampleNamesSkip = []

#If some blocks have failed to match previously using primary matching criteria, add them here to use secondary criteria
failedBlockFilenames = []

#If some blocks still fail to match, but have contents present in other blocks that do, then they may be discarded
discardBlockFilenames = []

#Ratio of red-channel values >=backgroundLevel in a patch, below which the patch will not be considered (default: 20%)
backgroundThreshold = 0.20

#Minimum red channel value [0, 255] for a location to contribute to the backgroundThreshold criteria (default: 5)
backgroundLevel = 5

#Block resolution
resolutionHeight, resolutionWidth = 2200, 2748

#Specify input/output patch size (symmetric)
patchSize = 400

#What weight should be used when overlaying annotation grid onto WSI
overlayWeight = 0.5

#How thick should the grid lines be when generating overlay images
gridThickness = 30


In [ ]:
#Raise the maximum image size for opencv; note that this can allow for decompression bomb DOS attacks if an untrusted image ends up as an input
import os 
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()

from IPython import get_ipython
if get_ipython().__class__.__name__ == 'ZMQInteractiveShell': jupyterNotebook = True
else: jupyterNotebook = False

import copy
import cv2
import glob
import imagej
import gc
import logging
import matlab.engine
import matplotlib.pyplot as plt
import natsort
import numpy as np
import pandas as pd
import psutil
import ray
import scyjava
import shutil
import time

from basicpy import BaSiC
from contextlib import nullcontext
from matplotlib import colors
from ray.util.multiprocessing import Pool
from scyjava import jimport
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.segmentation import find_boundaries

if jupyterNotebook: from tqdm.notebook import tqdm
else: from tqdm.auto import tqdm

#Start ImageJ with interactive GUI (required to use BaSiC plugin), legacy mode (required for running macros), and increased memory (no more than 80% system RAM; 25% for safety)
javaRAM = str(round(0.25*(psutil.virtual_memory().total / 1024**3)))
scyjava.config.add_option('-Xmx'+javaRAM+'g')

#Cannot replicate exact MATLAB behaviors in python, so start an engine to run them directly!
eng = matlab.engine.start_matlab()
    
#Pandas option to prevent terminal outputs
pd.set_option('future.no_silent_downcasting', True)

#Define logging levels and behaviors
logging.root.setLevel(logging.ERROR)
logging.raiseExceptions = False

#Determine size of small objects and holes to be removed according to threshold and patch size
removalSize = int(backgroundThreshold*(patchSize**2))

#Determine offset to avoid overlapping squares in grid visualization
gridThicknessOffset = gridThickness//2

#Setup label-to-color conversion
cmapClasses = colors.ListedColormap(['lime', 'blue', 'cyan', 'white', 'red', 'orange', 'magenta', 'yellow'])

#Ray actor for holding global progress in parallel sampling operations
@ray.remote(num_cpus=0)
class SamplingProgress_Actor:
    def __init__(self): self.current = 0.0
    def update(self): self.current += 1
    def getCurrent(self): return self.current

#OpenCV does not output sharp corners with its rectangle method...unless it's filled in
def rectangle(image, startPos, endPos, color):
    image = cv2.rectangle(image, startPos, endPos, color, -1)
    image = cv2.rectangle(image, (startPos[0]+gridThicknessOffset, startPos[1]+gridThicknessOffset), (endPos[0]-gridThicknessOffset, endPos[1]-gridThicknessOffset), (0, 0, 0), -1)
    return image


In [ ]:
#Setup output directory references
dir_out = './OUTPUTS/'
dir_out_processing = dir_out + '0_PROCESSING/'
dir_out_blockComparison = dir_out + '1_BLOCK_COMPARISON/'
dir_out_labels = dir_out + '2_GRID_LABELS/'
dir_uOut = dir_out + '3_UNCORRECTED/'
dir_oOut = dir_out + '4_IMAGEJ_CORRECTED/'
#dir_cOut = dir_out + '5_BASICPY_CORRECTED/'
dir_out_uWSI = dir_uOut + 'WSI/'
dir_out_uLabelsWSI = dir_uOut + 'WSI_LABELS/'
dir_out_uPatches = dir_uOut + 'PATCHES/'
dir_out_oWSI = dir_oOut + 'WSI/'
dir_out_oLabelsWSI = dir_oOut + 'WSI_LABELS/'
dir_out_oPatches = dir_oOut + 'PATCHES/'
#dir_out_cWSI = dir_cOut + 'WSI/'
#dir_out_cLabelsWSI = dir_cOut + 'WSI_LABELS/'
#dir_out_cPatches = dir_cOut + 'PATCHES/'

#Ensure output directories exist and are empty
if os.path.exists(dir_out): shutil.rmtree(dir_out)
os.makedirs(dir_out)
os.makedirs(dir_uOut)
#os.makedirs(dir_cOut)
os.makedirs(dir_out_uWSI)
#os.makedirs(dir_out_uLabelsWSI)
os.makedirs(dir_out_uPatches)
#os.makedirs(dir_out_cWSI)
#os.makedirs(dir_out_cLabelsWSI)
#os.makedirs(dir_out_cPatches)
os.makedirs(dir_out_oWSI)
#os.makedirs(dir_out_oLabelsWSI)
os.makedirs(dir_out_oPatches)
os.makedirs(dir_out_labels)
os.makedirs(dir_out_blockComparison)
os.makedirs(dir_out_processing)

#Derive how blocks should be cropped to allow for even patch splits
cropHeight = resolutionHeight-(int(resolutionHeight/patchSize)*patchSize)
cropWidth = resolutionWidth-(int(resolutionWidth/patchSize)*patchSize)
cropTop, cropLeft = cropHeight//2, cropWidth//2
cropBottom, cropRight = cropTop+(cropHeight%2), cropLeft+(cropWidth%2)

#Determine block size after cropping
blockHeight, blockWidth = resolutionHeight-cropTop-cropBottom, resolutionWidth-cropLeft-cropRight

#Determine how many patches are to be extracted from each block
numPatchesRow, numPatchesCol = blockHeight//patchSize, blockWidth//patchSize

#Setup an empty block image to use if a block doesn't have any patches to extract
emptyBlockImage = np.zeros((resolutionHeight, resolutionWidth))
emptyBlockImage3D = np.zeros((resolutionHeight, resolutionWidth, 3))

#Obtain list of sample names of available WSI .jpg images
if len(manualSamples) ==0: sampleNames = natsort.natsorted([os.path.splitext(name)[0] for name in os.listdir(dir_in_cWSI)])
else: sampleNames = manualSamples

#Remove any samples that are explicitly specified to be skipped
sampleNames = [sampleName for sampleName in sampleNames if sampleName not in sampleNamesSkip]

#Samples that are not wholly benign or malignant are presumed to be mixed
samplesMixed = natsort.natsorted([sampleName for sampleName in [sampleName for sampleName in sampleNames if sampleName not in samplesBenign] if sampleName not in samplesMalignant])

#Verify that annotations exist for each mixed sample; remove from processing queue if missing
missingAnnotations = [sampleName for sampleName in samplesMixed if not os.path.exists(dir_in_aWSI+sampleName+'.jpg') ]
for sampleName in missingAnnotations: 
    print('Error - Missing annotation for sample:', sampleName)
    sampleNames.remove(sampleName)
    samplesMixed.remove(sampleName)

#Verify that corrected WSI and uncorrected blocks exist for all samples to be processed; remove from processing queue if missing
for sampleName in samplesBenign+samplesMalignant+samplesMixed:
    removeSampleName = False
    if not os.path.exists(dir_in_cWSI+sampleName+'.jpg'): 
        print('Error - Missing WSI.jpg for sample:', sampleName)
        removeSampleName=True
    if not os.path.exists(dir_in_uBlocks+sampleName): 
        print('Error - Missing uncorrected blocks for sample:', sampleName)
        removeSampleName=True
    if removeSampleName:
        sampleNames.remove(sampleName)
        if sampleName in samplesBenign: samplesBenign.remove(sampleName)
        elif sampleName in samplesMalignant: samplesMalignant.remove(sampleName)
        elif sampleName in samplesMixed: samplesMixed.remove(sampleName)
    
#Create holding list for patch metadata; here to prevent overwrite if loop needs to be restarted
patchMetadata = []

In [ ]:
#For each sample extract annotations from stitched/corrected blocks, forming corrected/uncorrected WSI/patch sets
#for sampleName in tqdm(natsort.natsorted(sampleNames), total=len(sampleNames), desc='Samples', leave=True): 
for sampleName in tqdm(natsort.natsorted(sampleNames), total=len(sampleNames), desc='Samples', leave=True): 

    #Create output sub-folders/references for extracted patches and temporary processing
    #dir_out_cPatches_sample = dir_out_cPatches + sampleName + '/'
    dir_out_uPatches_sample = dir_out_uPatches + sampleName + '/'
    dir_out_oPatches_sample = dir_out_oPatches + sampleName + '/'
    dir_out_processing_sample = dir_out_processing + sampleName + '/'
    dir_sequenceFolder = (os.path.abspath(dir_out_processing_sample)+os.path.sep).replace(os.path.sep, '/')
    #if not os.path.exists(dir_out_cPatches_sample): os.makedirs(dir_out_cPatches_sample)
    if not os.path.exists(dir_out_uPatches_sample): os.makedirs(dir_out_uPatches_sample)
    if not os.path.exists(dir_out_oPatches_sample): os.makedirs(dir_out_oPatches_sample)
    if not os.path.exists(dir_out_processing_sample): os.makedirs(dir_out_processing_sample)
    
    #Determine filenames for uncorrected block images
    blockFilenames = natsort.natsorted(glob.glob(dir_in_uBlocks+sampleName+'/*.tif'))
    
    #########################################################################################
    ##Annotation extraction
    #########################################################################################
    
    #load corrected WSI
    imageWSI = cv2.imread(dir_in_cWSI + sampleName + '.jpg', cv2.IMREAD_UNCHANGED)
    grayImageWSI = cv2.cvtColor(imageWSI, cv2.COLOR_BGR2GRAY)
    
    #Remove areas/holes smaller than the backgroundThreshold*patch area where the outermost contour defines the foreground region
    fImageWSI = find_boundaries(grayImageWSI, mode='outer')
    fImageWSI = remove_small_objects(fImageWSI, min_size=removalSize)
    fImageWSI = remove_small_holes(fImageWSI, area_threshold=removalSize)
    fContours = cv2.findContours(fImageWSI.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]
    fImageWSI = np.zeros((fImageWSI.shape), dtype=int)
    _ = cv2.drawContours(fImageWSI, fContours, -1, 1, thickness=cv2.FILLED)
    del fContours
    
    #Load/process malignant position map
    if sampleName in samplesMalignant: mImageWSI = np.ones((imageWSI.shape[0], imageWSI.shape[1]), dtype=int)
    else: mImageWSI = np.zeros((imageWSI.shape[0], imageWSI.shape[1]), dtype=int)
    if sampleName in samplesMixed:
        annotationMap = (cv2.imread(dir_in_aWSI + sampleName + '.jpg', 0)>=230).astype('uint8')
        annotationMap = cv2.morphologyEx(annotationMap, cv2.MORPH_CLOSE, np.ones((19, 19), np.uint8))
        _ = cv2.drawContours(mImageWSI, cv2.findContours(annotationMap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0], -1, 1, thickness=cv2.FILLED)
        del annotationMap
    
    #########################################################################################
    
    
    #########################################################################################
    ##BaSiC correction as originally performed in ImageJ
    #########################################################################################
    
    #Split RGB into HSV; save H filenames as these will not be changed and basename for combined output labeling
    HFilenames, oBlockFilenames = [], []
    for fileLocation in blockFilenames:
        filename = os.path.basename(fileLocation)
        eng.eval("\
                I = imread('"+fileLocation+"');\
                hsv = rgb2hsv(I);\
                imwrite(hsv(:,:,1), '"+dir_out_processing_sample+'HChan_'+filename+"');\
                imwrite(hsv(:,:,2), '"+dir_out_processing_sample+'SChan_'+filename+"');\
                imwrite(hsv(:,:,3), '"+dir_out_processing_sample+'VChan_'+filename+"');\
                ", nargout=0)
        HFilenames.append(dir_out_processing_sample+'HChan_'+filename)
        oBlockFilenames.append(dir_sequenceFolder+'combined_'+filename)
    
    #Start ImageJ with interactive GUI (required to use BaSiC plugin)
    ij = imagej.init('./Fiji', add_legacy=True, mode='interactive')
    
    #Perform BaSiC flatfield and darkfield correction using ImageJ
    macro = ''
    macro += 'File.openSequence("'+dir_sequenceFolder+'", " filter=SChan");'
    macro += 'run("BaSiC ", "processing_stack='+sampleName+' flat-field=None dark-field=None shading_estimation=[Estimate shading profiles] shading_model=[Estimate both flat-field and dark-field] setting_regularisationparametes=Automatic temporal_drift=Ignore correction_options=[Compute shading and correct images] lambda_flat=0.50 lambda_dark=0.50");'
    macro += 'close("Flat-field:'+sampleName+'");'
    macro += 'close("Dark-field:'+sampleName+'");'
    macro += 'selectImage("Corrected:'+sampleName+'");'
    macro += 'run("Image Sequence... ", "select='+dir_sequenceFolder+' dir='+dir_sequenceFolder+' format=TIFF name=correctedSChan start=1 digits=4");'
    macro += 'close("Corrected:'+sampleName+'");'
    macro += 'close("'+sampleName+'");'
    macro += 'File.openSequence("'+dir_sequenceFolder+'", " filter=VChan");'
    macro += 'run("BaSiC ", "processing_stack='+sampleName+' flat-field=None dark-field=None shading_estimation=[Estimate shading profiles] shading_model=[Estimate both flat-field and dark-field] setting_regularisationparametes=Automatic temporal_drift=Ignore correction_options=[Compute shading and correct images] lambda_flat=0.50 lambda_dark=0.50");'
    macro += 'close("Flat-field:'+sampleName+'");'
    macro += 'close("Dark-field:'+sampleName+'");'
    macro += 'selectImage("Corrected:'+sampleName+'");'
    macro += 'run("Image Sequence... ", "select='+dir_sequenceFolder+' dir='+dir_sequenceFolder+' format=TIFF name=correctedVChan start=1 digits=4");'
    macro += 'close("Corrected:'+sampleName+'");'
    macro += 'close("'+sampleName+'");'
    macro += 'close("Log");'
    _ = ij.py.run_macro(macro)
    
    #Shutdown ImageJ environment and clear RAM used for JVM (which otherwise balloons until OOM outside python)
    ij.dispose()
    
    #Alternatively may be able to just manually run garbage collection in JVM
    #_ = ij.py.run_macro("Garbage Collection")
    
    #or use new_instance tag...documentation is rather sparse
    #ij.getContext().dispose()
    #ij = imagej.init('./Fiji', add_legacy=True, mode='interactive',  new_instance=True))
    
    #Obtain ordered list of the S and V filenames
    SFilenames = natsort.natsorted(glob.glob(dir_sequenceFolder+'correctedSChan*.tif'))
    VFilenames = natsort.natsorted(glob.glob(dir_sequenceFolder+'correctedVChan*.tif'))

    #Combine original H with corrected S and V channels, then combine/convert/export combined RGB images
    for blockIndex, oBlockFilename in enumerate(oBlockFilenames):
        eng.eval("\
                    I = zeros("+str(resolutionHeight)+","+str(resolutionWidth)+",3);\
                    I(:,:,1) = imread('"+HFilenames[blockIndex]+"');\
                    I(:,:,2) = imread('"+SFilenames[blockIndex]+"');\
                    I(:,:,3) = imread('"+VFilenames[blockIndex]+"');\
                    I = I./255;\
                    imwrite(hsv2rgb(I), '"+oBlockFilename+"');\
                    ", nargout=0)
    
    #Load original corrected sample block images
    oBlockImages = [cv2.imread(filename, cv2.IMREAD_UNCHANGED) for filename in oBlockFilenames]
    
    #Delete temporary sample processing folder   
    if os.path.exists(dir_out_processing_sample): shutil.rmtree(dir_out_processing_sample)
    
    #Load uncorrected sample block images
    uBlockImages = [cv2.imread(filename, cv2.IMREAD_UNCHANGED) for filename in blockFilenames]
    
    #Extract WSI block-level dimensions, asuming block filenames are 1 indexed and the last indexed is the bottom right
    _, _, _, row, column = os.path.basename(blockFilenames[-1]).split('.')[0].split('_')
    blocksY, blocksX = int(row.split('R')[1]), int(column.split('C')[1])
    
    #########################################################################################
    
    
    #########################################################################################
    ##BaSiC correction as implemented in DDSM-GUI
    #########################################################################################
    
    #Create corrected versions of block images
    #H, S, V = np.split(np.asarray([cv2.cvtColor(image.astype('float32'), cv2.COLOR_BGR2HSV_FULL) for image in uBlockImages]), 3, -1)
    #H, S, V = H[..., 0], S[..., 0], V[..., 0]
    #V = BaSiC(fitting_mode='approximate', optimization_tol=1e-6, reweighting_tol=1e-3, sort_intensity=True).fit_transform(V)
    #V = (V / V.max())*255.0
    #cBlockImages = np.asarray([np.round(cv2.cvtColor(image, cv2.COLOR_HSV2BGR_FULL)).astype('uint8') for image in np.stack([H, S, V], -1)])
    #del H, S, V
    #gc.collect()
    
    #########################################################################################
    
    
    #########################################################################################
    ##Extraction and transfer
    #########################################################################################
    
    #Extract annotations for blocks and visualize for manually conducted comparitive audit
    mBlockImages, fBlockImages, sBlockImage = [], [], None
    for index, oBlockImage in tqdm(enumerate(oBlockImages), total=len(oBlockImages), desc='Blocks', leave=False): 
        
        #If the block has patches that would be extracted (meeting the backgroundLevel, backgroundThreshold criteria, and not manually discarded) perform matching
        if os.path.basename(blockFilenames[index]) in discardBlockFilenames: 
            validBlock = False
            oBlockImages[index] = copy.deepcopy(emptyBlockImage3D)
        else: 
            validBlock = oBlockImage[cropTop:-cropBottom, cropLeft:-cropRight, :]
            validBlock = validBlock.reshape(numPatchesRow, patchSize, numPatchesCol, patchSize, validBlock.shape[2]).swapaxes(1,2)
            validBlock = validBlock.reshape(-1, validBlock.shape[2], validBlock.shape[3], validBlock.shape[4])
            validBlock = np.max(np.mean(validBlock[:,:,:,2]>=backgroundLevel, axis=(1,2))) > backgroundThreshold
        
        if validBlock:
            
            #Find/extract block from stitched/corrected WSI, extracting its annotations, use alternate criteria if match failed previously
            #Allowable metrics: cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR, cv2.TM_CCORR_NORMED, cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED
            if os.path.basename(blockFilenames[index]) in failedBlockFilenames: matchMap = cv2.matchTemplate(grayImageWSI, cv2.cvtColor(oBlockImage, cv2.COLOR_BGR2GRAY), cv2.TM_CCORR_NORMED)
            else: matchMap = cv2.matchTemplate(grayImageWSI, cv2.cvtColor(oBlockImage, cv2.COLOR_BGR2GRAY), cv2.TM_CCOEFF_NORMED)
            startRow, startColumn = np.unravel_index(np.argmax(matchMap), matchMap.shape)
            sBlockImage = imageWSI[startRow:startRow+resolutionHeight, startColumn:startColumn+resolutionWidth]
            mBlockImages.append(mImageWSI[startRow:startRow+resolutionHeight, startColumn:startColumn+resolutionWidth])
            fBlockImages.append(fImageWSI[startRow:startRow+resolutionHeight, startColumn:startColumn+resolutionWidth])
            
            #If not an exact match, create side-by-side comparison of the block images for visual audit
            if np.sum(np.abs(oBlockImage-sBlockImage)) != 0:
                #f = plt.figure(figsize=(11,9))
                f = plt.figure(figsize=(16,5))
                plt.suptitle('Sample ' + sampleName)
                #ax = plt.subplot2grid((2,2), (0,0))
                ax = plt.subplot2grid((1,3), (0,0))
                im = ax.imshow(cv2.cvtColor(sBlockImage, cv2.COLOR_BGR2RGB), interpolation='none')
                ax.set_title('Corrected Stitched WSI Extraction')
                #ax = plt.subplot2grid((2,2), (0,1))
                ax = plt.subplot2grid((1,3), (0,1))
                im = ax.imshow(cv2.cvtColor(oBlockImage, cv2.COLOR_BGR2RGB), interpolation='none')
                ax.set_title('ImageJ Basic Corrected')
                #ax = plt.subplot2grid((2,2), (1,0))
                #im = ax.imshow(cv2.cvtColor(cBlockImages[index], cv2.COLOR_BGR2RGB), interpolation='none')
                #ax.set_title('BasicPy Corrected')
                #ax = plt.subplot2grid((2,2), (1,1))
                ax = plt.subplot2grid((1,3), (0,2))
                im = ax.imshow(cv2.cvtColor(uBlockImages[index], cv2.COLOR_BGR2RGB), interpolation='none')
                ax.set_title('Uncorrected')
                f.tight_layout()
                filenameOutput = dir_out_blockComparison + os.path.basename(blockFilenames[index]).split('.')[0] + '.tif'
                plt.savefig(filenameOutput)
                plt.close()
        else: 
            mBlockImages.append(copy.deepcopy(emptyBlockImage))
            fBlockImages.append(copy.deepcopy(emptyBlockImage))
    del grayImageWSI, imageWSI, matchMap, oBlockImage, sBlockImage, validBlock
    gc.collect()
    
    #Crop block images and matched pixel-level malignant and foreground maps
    uBlockImages = np.asarray(uBlockImages)[:, cropTop:-cropBottom, cropLeft:-cropRight, :]
    #cBlockImages = np.asarray(cBlockImages)[:, cropTop:-cropBottom, cropLeft:-cropRight, :]
    oBlockImages = np.asarray(oBlockImages)[:, cropTop:-cropBottom, cropLeft:-cropRight, :]
    mBlockImages = np.asarray(mBlockImages)[:, cropTop:-cropBottom, cropLeft:-cropRight]
    fBlockImages = np.asarray(fBlockImages)[:, cropTop:-cropBottom, cropLeft:-cropRight]
    
    #Form new complete WSI, malignant, and foreground map
    height, width = blocksY*blockHeight,  blocksX*blockWidth
    uImageWSI = uBlockImages.reshape(blocksY, blocksX, blockHeight, blockWidth, 3)
    uImageWSI = uImageWSI.swapaxes(1,2).reshape(height, width, 3)
    #cImageWSI = cBlockImages.reshape(blocksY, blocksX, blockHeight, blockWidth, 3)
    #cImageWSI = cImageWSI.swapaxes(1,2).reshape(height, width, 3)
    oImageWSI = oBlockImages.reshape(blocksY, blocksX, blockHeight, blockWidth, 3)
    oImageWSI = oImageWSI.swapaxes(1,2).reshape(height, width, 3)
    mImageWSI = mBlockImages.reshape(blocksY, blocksX, blockHeight, blockWidth)
    mImageWSI = mImageWSI.swapaxes(1,2).reshape(height, width)
    fImageWSI = fBlockImages.reshape(blocksY, blocksX, blockHeight, blockWidth)
    fImageWSI = fImageWSI.swapaxes(1,2).reshape(height, width)
    
    #Padding should not be needed given the cropping process
    #Pad (as symmetrically as possible) for an even division by the configured patch size
    #padHeight = (int(np.ceil(uImageWSI.shape[0]/patchSize))*patchSize)-uImageWSI.shape[0]
    #padWidth = (int(np.ceil(uImageWSI.shape[1]/patchSize))*patchSize)-uImageWSI.shape[1]
    #padTop, padLeft = padHeight//2, padWidth//2
    #padBottom, padRight = padTop+(padHeight%2), padLeft+(padWidth%2)
    #if padTop !=0 or padLeft !=0 or padBottom !=0 or padRight !=0: print('Warning - Required padding for even patch extraction was non-zero for sample:', sampleName)
    #uImageWSI = np.pad(uImageWSI, ((padTop, padBottom), (padLeft, padRight), (0, 0)))
    #cImageWSI = np.pad(cImageWSI, ((padTop, padBottom), (padLeft, padRight), (0, 0)))
    #oImageWSI = np.pad(oImageWSI, ((padTop, padBottom), (padLeft, padRight), (0, 0)))
    #mImageWSI = np.pad(mImageWSI, ((padTop, padBottom), (padLeft, padRight)))
    #fImageWSI = np.pad(fImageWSI, ((padTop, padBottom), (padLeft, padRight)))
    
    #Determine benign map as foreground locations that are not malignant
    bImageWSI = ((fImageWSI-mImageWSI)>0).astype('uint8')
    
    #Export the new WSI
    writeSuccess = cv2.imwrite(dir_out_uWSI+sampleName+'.tif', uImageWSI, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    #writeSuccess = cv2.imwrite(dir_out_cWSI+sampleName+'.tif', cImageWSI, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    writeSuccess = cv2.imwrite(dir_out_oWSI+sampleName+'.tif', oImageWSI, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    
    #Create an empty array and grayscale copies (in BGR) of the WSI to generate/visualize label/annotation grid
    patchGrid = np.zeros(uImageWSI.shape, dtype=np.uint8)
    #uLabelsImageWSI = cv2.cvtColor(cv2.cvtColor(uImageWSI, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)
    #cLabelsImageWSI = cv2.cvtColor(cv2.cvtColor(cImageWSI, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)
    #oLabelsImageWSI = cv2.cvtColor(cv2.cvtColor(oImageWSI, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)
    
    #Split the WSI and annotation map into flat lists of patches
    numPatchesRowTotal, numPatchesColTotal = numPatchesRow*blocksY, numPatchesCol*blocksX
    uImageWSI = uImageWSI.reshape(numPatchesRowTotal, patchSize, numPatchesColTotal, patchSize, uImageWSI.shape[2]).swapaxes(1,2)
    uImageWSI = uImageWSI.reshape(-1, uImageWSI.shape[2], uImageWSI.shape[3], uImageWSI.shape[4])
    #cImageWSI = cImageWSI.reshape(numPatchesRow, patchSize, numPatchesCol, patchSize, cImageWSI.shape[2]).swapaxes(1,2)
    #cImageWSI = cImageWSI.reshape(-1, cImageWSI.shape[2], cImageWSI.shape[3], cImageWSI.shape[4])
    oImageWSI = oImageWSI.reshape(numPatchesRowTotal, patchSize, numPatchesColTotal, patchSize, oImageWSI.shape[2]).swapaxes(1,2)
    oImageWSI = oImageWSI.reshape(-1, oImageWSI.shape[2], oImageWSI.shape[3], oImageWSI.shape[4])
    mImageWSI = mImageWSI.reshape(numPatchesRowTotal, patchSize, numPatchesColTotal, patchSize).swapaxes(1,2)
    mImageWSI = mImageWSI.reshape(-1, mImageWSI.shape[2], mImageWSI.shape[3])
    fImageWSI = fImageWSI.reshape(numPatchesRowTotal, patchSize, numPatchesColTotal, patchSize).swapaxes(1,2)
    fImageWSI = fImageWSI.reshape(-1, fImageWSI.shape[2], fImageWSI.shape[3])
    bImageWSI = bImageWSI.reshape(numPatchesRowTotal, patchSize, numPatchesColTotal, patchSize).swapaxes(1,2)
    bImageWSI = bImageWSI.reshape(-1, bImageWSI.shape[2], bImageWSI.shape[3])
    
    #Export and annotate uncorrected and corrected patches
    patchIndex = 0
    for index, (rowNum, colNum) in tqdm(enumerate(np.ndindex((numPatchesRowTotal,numPatchesColTotal))), total=numPatchesRowTotal*numPatchesColTotal, desc='Patches', leave=False):

        #Extract uncorrected and corrected patches 
        #uImagePatch, cImagePatch, oImagePatch = uImageWSI[index], cImageWSI[index], oImageWSI[index]
        uImagePatch, oImagePatch = uImageWSI[index], oImageWSI[index]

        #Export patches that would qualify for classification if uncorrected
        if (np.mean(uImagePatch[:,:,2] >= backgroundLevel) > backgroundThreshold):

            #Determine pixel location and setup name for exported patch image/metadata
            locationRow, locationColumn = rowNum*patchSize, colNum*patchSize
            patchName = sampleName+'_'+str(patchIndex)+'_'+str(locationRow)+'_'+str(locationColumn)
            patchFilename = 'PS' + patchName + '.tif'

            #Extract malignant, benign, and foreground patch data
            mImagePatch, bImagePatch, fImagePatch,  = mImageWSI[index], bImageWSI[index], fImageWSI[index]
            
            #Determine if patch contains any malignant/benign/edge/boundary locations
            labelMalignant = True if np.sum(mImagePatch) > 0 else False
            labelBenign = True if np.sum(bImagePatch) > 0 else False
            labelEdge = True if (np.sum(1-fImagePatch) > 0) else False
            labelBoundary = True if (labelMalignant and labelBenign) else False
            
            #Should not have non-malignant and boundary labels together ('Blue' or 'White')
            if not labelMalignant and labelBoundary and not labelEdge: print('Blue (Benign, Boundary) label should not have been generated:        ', patchName)
            if not labelMalignant and labelBoundary and labelEdge: print('White (Benign, Boundary, Edge) label should not have been generated: ', patchName)
            
            #Add patch annotation to grid overlay
            patchColor  = (np.asarray(cmapClasses(int(str(np.uint8(labelMalignant))+str(np.uint8(labelEdge))+str(np.uint8(labelBoundary)), 2))[:3])*255).astype('uint8').tolist()
            patchGrid = rectangle(patchGrid, (locationColumn, locationRow), (locationColumn+patchSize, locationRow+patchSize), patchColor)
            
            #Convert annotations to text labels
            labelMalignant = 'T' if labelMalignant else 'N'
            labelEdge = 'E' if labelEdge else 'N'
            labelBoundary = 'B' if labelBoundary else 'N'
            patchAnnotation = [labelMalignant, labelEdge, labelBoundary]
            
            #Export patches
            writeSuccess = cv2.imwrite(dir_out_uPatches_sample+patchFilename, uImagePatch, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
            #writeSuccess = cv2.imwrite(dir_out_cPatches_sample+patchFilename, cImagePatch, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
            writeSuccess = cv2.imwrite(dir_out_oPatches_sample+patchFilename, oImagePatch, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))

            #Store metadata
            patchMetadata.append([patchName, patchIndex, locationRow, locationColumn]+patchAnnotation)
            
            #Increase patch index
            patchIndex += 1
    
    #Overlay annotation grid on top of WSI and store to disk
    #uLabelsImageWSI = cv2.cvtColor(cv2.addWeighted(uLabelsImageWSI, 1.0, patchGrid, overlayWeight, 0.0), cv2.COLOR_RGB2BGR)
    #cLabelsImageWSI = cv2.cvtColor(cv2.addWeighted(cLabelsImageWSI, 1.0, patchGrid, overlayWeight, 0.0), cv2.COLOR_RGB2BGR)
    #oLabelsImageWSI = cv2.cvtColor(cv2.addWeighted(oLabelsImageWSI, 1.0, patchGrid, overlayWeight, 0.0), cv2.COLOR_RGB2BGR)
    patchGrid = cv2.cvtColor(np.dstack((patchGrid, np.uint8((np.sum(patchGrid, axis=-1) > 0)*255))), cv2.COLOR_RGBA2BGRA)
    #writeSuccess = cv2.imwrite(dir_out_uLabelsWSI+'overlaid_labelGrid_'+sampleName+'.tif', uLabelsImageWSI, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    #writeSuccess = cv2.imwrite(dir_out_cLabelsWSI+'overlaid_labelGrid_'+sampleName+'.tif', cLabelsImageWSI, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    #writeSuccess = cv2.imwrite(dir_out_oLabelsWSI+'overlaid_labelGrid_'+sampleName+'.tif', oLabelsImageWSI, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    writeSuccess = cv2.imwrite(dir_out_labels+'labelGrid_'+sampleName+'.tif', patchGrid, params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    
    #Export all patch metadata recorded to this time (perform after each sample in case of crash)
    dataPrintout = pd.DataFrame(np.asarray(patchMetadata))
    dataPrintout.columns=['Sample', 'Index', 'Row', 'Column', 'Label', 'Edge', 'Boundary']
    dataPrintout.to_csv(dir_out_uPatches+'metadata_patches.csv', index=False)
    #dataPrintout.to_csv(dir_out_cPatches+'metadata_patches.csv', index=False)
    dataPrintout.to_csv(dir_out_oPatches+'metadata_patches.csv', index=False)
    
    #Delete larger objects that are no longer needed and run garbage collection
    del patchGrid, fImageWSI, mImageWSI, bImagePatch, dataPrintout
    gc.collect()
    


In [ ]:
#Extract sorted list of variables in RAM (debugging/optimization)
#Ref: https://stackoverflow.com/questions/40993626/list-memory-usage-in-ipython-and-jupyter
#import sys
#ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
#sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)